In [240]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import heapq

In [241]:
size = (30, 50)
obstacles = [(5, 5, 10, 10), (20, 30, 25, 45), (15, 0, 20, 5)]
start = (0, 0)
goal = (29, 40)

In [242]:
class Node:
  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.parent_node = None

In [243]:
def is_within_obstacles(point, obstacles):
  for (ox, oy, ex, ey) in obstacles:
    if ox <= point[0] <= ex and oy <= point[1] <= ey:
      return True
  return False

def distance(point1, point2):
  return np.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)

def nearest_node(nodes, random_point):
  return min(nodes, key=lambda node: distance((node.x, node.y), random_point))

def steer(from_node, to_point, step_size=1):
  if distance((from_node.x, from_node.y), to_point) < step_size:
    return Node(to_point[0], to_point[1])
  else:
    theta = np.arctan2(to_point[1]-from_node.y, to_point[0]-from_node.x)
    return Node(from_node.x + step_size*np.cos(theta), from_node.y + step_size*np.sin(theta))

""" Check if the path between node1 and node2 is valid by interpolating points along the way. """
def is_valid_path(node1, node2, obstacles):
  steps = int(distance((node1.x, node1.y), (node2.x, node2.y))/0.5)  # Smaller steps for more accuracy

  for i in range(1, steps + 1):
    inter_x = node1.x + i*(node2.x-node1.x)/steps
    inter_y = node1.y + i*(node2.y-node1.y)/steps

    if is_within_obstacles((inter_x, inter_y), obstacles):
      return False
  return True


def plot(nodes=None, path=None):
    fig, ax = plt.subplots()
    if nodes:
      for node in nodes:
          if node.parent_node:
              plt.plot([node.x, node.parent_node.x], [node.y, node.parent_node.y], "g-", linewidth=0.5)
    for (ox, oy, ex, ey) in obstacles:
        ax.add_patch(plt.Rectangle((ox, oy), ex-ox, ey-oy, color="red"))
    if path:
        plt.plot([node.x for node in path], [node.y for node in path], "b-", linewidth=2)  # Highlight path in blue
    plt.plot(start[0], start[1], "bo")  # Start
    plt.plot(goal[0], goal[1], "ro")  # Goal
    plt.grid(True)
    plt.show()

In [244]:
def rrt(step_size=5, max_nodes=10000):
    nodes = [Node(start[0], start[1])]
    while len(nodes) < max_nodes:
        random_point = (random.randint(0, size[0] - 1), random.randint(0, size[1] - 1))
        if is_within_obstacles(random_point, obstacles):
            continue
        nearest = nearest_node(nodes, random_point)
        new_node = steer(nearest, random_point, step_size)
        if not is_within_obstacles((new_node.x, new_node.y), obstacles) and is_valid_path(nearest, new_node, obstacles):
            new_node.parent_node = nearest
            nodes.append(new_node)
            if distance((new_node.x, new_node.y), goal) <= 2:#step_size:
                return nodes, new_node
    return nodes, None  # Return None if max_nodes reached without finding a path

In [245]:
def create_initial_population(population_size=10, start=(0,0), goal=(29, 40) ,plot_paths = False):
  population = []
  # solution = []
  for i in range(population_size):
    nodes, final_node = rrt()
    path = []
    if final_node:
        while final_node.parent_node:
            path.append(final_node)
            final_node = final_node.parent_node
        path.append(final_node)
        path.reverse()
    if plot_paths:
      print(path)
      plot(nodes, path)
    population.append(path)
  return population

In [246]:
def fitness_function(path):
    coordinates = [(node.x, node.y) for node in path]
    wt = 2
    euc_dist = 0
    for i in range(len(coordinates) - 1):
        x1, y1 = coordinates[i]
        x2, y2 = coordinates[i+1]
        dist_ = math.sqrt((x2-x1)**2 +(y2-y1)**2)
        euc_dist += dist_
        if euc_dist > 0:
            F = 1 / (wt*euc_dist)
        else:
            F = float('inf')
    return euc_dist, F

In [247]:
def calcualte_fitness_of_population(population):
  fitness = []
  for index, path in enumerate(population):
    euc_dist, F = fitness_function(path)
    fitness.append(F)
  return fitness

In [248]:
def selection(fitness, population):
  P1 = []
  P2 = []
  while(fitness):
    max_fitness = max(fitness)
    index = fitness.index(max_fitness)
    fitness.pop(index)
    P1.append(population.pop(index))

    l = len(fitness)
    i = random.randint(0, l-1)
    fitness.pop(i)
    P2.append(population.pop(i))
  return P1, P2

In [249]:
def elimination(fitness, population):
  num_eliminations = int(0.1*len(fitness))
  for i in range(0, num_eliminations):
    min_fitness = min(fitness)
    index = fitness.index(min_fitness)
    fitness.pop(index)
    population.pop(index)
  return fitness, population


In [250]:
population = create_initial_population(population_size=200, plot_paths=False)

In [251]:
fitness = calcualte_fitness_of_population(population)

In [252]:
fitness, population = elimination(fitness, population)

In [253]:
P1, P2 = selection(fitness, population)